# stim 表征数据划分数据集

In [1]:
import numpy as np
import scipy.io
from tqdm import tqdm
import argparse
import os

In [2]:
'''参数设置'''
subject='subj01'  # 受试者
use_stim = 'ave'  # ave / each
featname = 'dpt'  # 表征 init_latent /  c / dpt_emb0~3

In [3]:
# 数据路径
basedir = '../data/nsdfeat/'
featdir = f'{basedir}/{featname}/'
nsd_expdesign = scipy.io.loadmat('../data/nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

#　保存路径
savedir = f'{basedir}/{subject}_feat/'
os.makedirs(savedir, exist_ok=True)

In [4]:
# Note that most of them are 1-base index!
# This is why I subtract 1
sharedix = nsd_expdesign['sharedix'] -1 
print(f'测试集大小：{len(sharedix[0])}')
if use_stim == 'ave':
    stims = np.load(f'../data/stim/{subject}/{subject}_stims_ave.npy') # 平均值 30000/3 = 10000 (9000+1000)
else: # Each
    stims = np.load(f'../data/stim/{subject}/{subject}_stims.npy')  # 每人30000次刺激

测试集大小：1000


In [5]:
# 特征数据和索引信息
feats = []
tr_idx = np.zeros(len(stims))

# 遍历刺激数据，加载图像特征数据，并更新索引
for idx, stim in tqdm(enumerate(stims)): 
    # if idx % 1000 == 0:
    #     print(idx,stim)
    if stim in sharedix:  # 测试数据
        tr_idx[idx] = 0
    else:  # 训练数据
        tr_idx[idx] = 1
    # 读取对应的刺激图像的表征数据
    feat = np.load(f'{featdir}/{stim:06}.npy')
    feats.append(feat)
feats = np.stack(feats)    



# 训练&测试 图像表征集合
feats_tr = feats[tr_idx==1,:]
feats_te = feats[tr_idx==0,:]

# 保存subj的刺激对应 tr_or_te
np.save(f'../data/stim/{subject}/{subject}_stims_tridx.npy',tr_idx)

# 保存subj的tr和te刺激图像的表征数据
np.save(f'{savedir}/{subject}_{use_stim}_{featname}_tr.npy',feats_tr)
np.save(f'{savedir}/{subject}_{use_stim}_{featname}_te.npy',feats_te)

10000it [11:06, 15.00it/s]


In [6]:
print(feats_tr.shape)
print(feats_te.shape)

(9000, 1, 384, 384)
(1000, 1, 384, 384)


In [107]:
# init_latent
# each
## tr: (27000, 6400)
## te: (3000, 6400)

# ave   
## tr: (9000, 6400)
## te: (1000, 6400)